In [2]:
import pandas as pd
import tmdbsimple as ts
from copy import deepcopy
import omdb 
from omdb import OMDBClient

ts.API_KEY='YOUR_API_KEY'

In [2]:
movies=pd.read_csv('IMDB movies.csv')

C:\Users\Harman\.conda\envs\PDSH\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Drop irrelevant columns

movies.drop(columns=['usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics','duration','language','date_published','budget','title'],inplace=True)

#Remove nulls
movies.dropna(inplace=True)
movies.shape

(78344, 12)

In [4]:
movies.columns.tolist()

['imdb_title_id',
 'original_title',
 'year',
 'genre',
 'country',
 'director',
 'writer',
 'production_company',
 'actors',
 'description',
 'avg_vote',
 'votes']

In [55]:
movies.head(5)

,imdb_title_id,original_title,year,genre,director,writer,production_company,actors,description,avg_vote,votes
0,tt0000009,Miss Jerry,1894,Romance,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589
2,tt0001892,Den sorte drøm,1911,Drama,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188
3,tt0002101,Cleopatra,1912,"Drama, History",Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy","Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237


In [59]:
#Get a list of imdb title ids
m_list=movies.imdb_title_id.to_list()

#Copy to another list
m_curr=deepcopy(m_list)

In [110]:
#Create a new dataframe to populate new_movie.csv
df_col=['imdb_id','tmdb_id', 'original_language', 'original_title', 'adult', 'overview', 'poster_path', 'popularity']
df_new=pd.DataFrame(columns=df_col)

In [159]:
#Populate df using TMDB api

for _,item in enumerate(m_list):
    res=ts.Find(id=item).info(external_source='imdb_id')['movie_results']
    
    #Deleting an id is equivalent to mark as done
    m_curr.pop(0)
    
    #If there is a match
    if res:
        res=res[0]
        row=dict.fromkeys(df_col,'')
    
        row['imdb_id']=item
        row['tmdb_id']=res['id']
        row['original_language']=res['original_language']
        row['original_title']=res['original_title']
        row['adult']=res['adult']
        row['overview']=res['overview']
        if res['poster_path']:
            row['poster_path']='http://image.tmdb.org/t/p/original'+ res['poster_path']
        row['popularity']=res['popularity']
        
        df_new=df_new.append(row,ignore_index=True)
                

In [160]:
#Append this df to new_movie.csv
df_new.to_csv('new_movie.csv', mode='a',header=False,index=False)

In [155]:
#Incase you stop the loop
m_curr.insert(0,item)
m_list=deepcopy(m_curr)
df_new=pd.DataFrame(columns=df_col)

In [156]:
#Check this before starting the loop again
assert(len(m_list) == len(m_curr))

True

In [3]:
#OMDB api operations
ob = OMDBClient(apikey='1eedb1c5')
res = ob.get(imdbid='tt9914942')

In [44]:
res

{'title': 'La vida sense la Sara Amat',
 'year': '2019',
 'rated': 'N/A',
 'released': '05 Feb 2020',
 'runtime': '74 min',
 'genre': 'Drama',
 'director': 'Laura Jou',
 'writer': 'Coral Cruz, Pep Puig (novel)',
 'actors': 'Maria Morera Colomer, Biel Rossell Pelfort, Isaac Alcayde, Lluís Altés',
 'plot': 'Pep, a 13-year-old boy, is in love with a girl from his grandparents village, Sara Amat. One summer night Sara disappears without a trace. After a few hours, Pep finds her hiding in his room.',
 'language': 'Catalan',
 'country': 'Spain',
 'awards': '2 wins & 1 nomination.',
 'poster': 'https://m.media-amazon.com/images/M/MV5BNDdhY2ZlZTItMTQ1Yy00N2ZlLTk3ZWEtZDc1ZGZiNzZlNmJjXkEyXkFqcGdeQXVyMTA0MjU0Ng@@._V1_SX300.jpg',
 'ratings': [{'source': 'Internet Movie Database', 'value': '6.8/10'}],
 'metascore': 'N/A',
 'imdb_rating': '6.8',
 'imdb_votes': '79',
 'imdb_id': 'tt9914942',
 'type': 'movie',
 'dvd': 'N/A',
 'box_office': 'N/A',
 'production': 'N/A',
 'website': 'N/A',
 'response': '

In [6]:
#Reading dump files from TMDB
df=pd.read_json('movie_ids_10_19_2020.json',lines=True)
df.head()

,adult,id,original_title,popularity,video
0,False,3924,Blondie,2.726,False
1,False,6124,Der Mann ohne Namen,0.840,False
2,False,8773,L'amour à vingt ans,3.243,False
3,False,21624,Jatts in Golmaal,0.600,False
4,False,25449,New World Disorder 9: Never Enough,1.542,False


In [8]:
#Get movies for year 2020 till current date

res=ts.Discover().movie(sort_by='primary_release_date.asc',primary_release_year='2020',primary_release_date_gte='2020-01-01',primary_release_date_lte='2020-10-27')


In [23]:
#Initialise a temporary list to store movie ids
id_list=[]

In [33]:
for i in range(1,500):
    res=ts.Discover().movie(page=i,sort_by='primary_release_date.asc',primary_release_year='2020',primary_release_date_gte='2020-01-01',primary_release_date_lte='2020-10-27')

    temp=pd.json_normalize(res['results']).id.tolist()
    
    id_list +=temp
    
    if i%100 == 0:
        print(100)

100
100


In [61]:
#Convert the list to a pandas dataframe
df=pd.DataFrame(data=id_list,columns=['id'])
df.id=df.id.astype('str')

#Filter out duplicates
df.drop_duplicates(keep="first",inplace=True)


,id
0,747527
1,742073
2,740646
3,733888
4,727158


In [65]:
#Read old ids and filter out the common ids
df_latest=pd.read_csv('movie_data.csv')
old_list=df_latest.tmdb_id.astype('str').tolist()
movie_list=df[~df.id.isin(old_list)].id.tolist()

C:\Users\Harman\.conda\envs\PDSH\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
##Generate data for all of the newly created movie ids

In [91]:
for i,tmdb_id in enumerate(movie_list):
    try:
        
        temp=pd.json_normalize(ts.Movies(id=tmdb_id).info(append_to_response='credits'))
        df_new=df_new.append(temp,ignore_index=True)
        if i%1000 == 0:
            print('True')
    except:
        pass

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [94]:
#Drop irrelavant columns
df_new.drop(columns=['belongs_to_collection.id','belongs_to_collection','backdrop_path', 'belongs_to_collection.name', 'belongs_to_collection.poster_path', 'belongs_to_collection.backdrop_path','status'],inplace=True)

In [95]:
#Save to disk
df_new.to_csv('tmdb_2020.csv',index=False)